# Input Cookies

In [ ]:
cookies = {
    '__lt__cid': '5b5d3611-4246-47da-a426-fab19d5167b1',
    '__lt__sid': '34f70d0c-1305710a',
    '_fbp': 'fb.2.1759160179308.71906476786345939',
    '_ga': 'GA1.4.274738429.1759160179',
    '_ga': 'GA1.1.274738429.1759160179',
    '_ga_JSJ77CR4LH': 'GS2.1.s1759715634$o14$g1$t1759715659$j35$l0$h0',
    '_gat_UA-138055626-1': '1',
    '_gcl_au': '1.1.688477002.1759160179',
    '_gid': 'GA1.3.1523821141.1759490953',
    '_gid': 'GA1.4.1523821141.1759490953',
    '_yjsu_yjad': '1759160179.3323d0fc-fe73-4092-9833-2d4fd12814f3',
    'has_redirected': 'FALSE',
    'OptanonAlertBoxClosed': '2025-09-29T15:39:32.295Z',
    'OptanonAlertBoxClosed': '2025-10-05T15:11:36.675Z',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Mon+Oct+06+2025+09%3A53%3A54+GMT%2B0800+(%E9%A6%99%E6%B8%AF%E6%A8%99%E6%BA%96%E6%99%82%E9%96%93)&version=6.33.0&isIABGlobal=false&hosts=&consentId=33f25ca8-45d1-4b3b-991f-8ad080b26b4e&interactionCount=2&landingPath=NotLandingPage&groups=C0004%3A0%2CC0001%3A1%2CC0002%3A0&AwaitingReconsent=false&geolocation=HK%3B',
    'OptanonConsent': 'isGpcEnabled=0&datestamp=Mon+Oct+06+2025+09%3A54%3A20+GMT%2B0800+(%E9%A6%99%E6%B8%AF%E6%A8%99%E6%BA%96%E6%99%82%E9%96%93)&version=202303.2.0&isIABGlobal=false&hosts=&consentId=33f25ca8-45d1-4b3b-991f-8ad080b26b4e&interactionCount=2&landingPath=NotLandingPage&groups=C0004%3A0%2CC0001%3A1%2CC0002%3A0&AwaitingReconsent=false&geolocation=HK%3B',
    'q_i_t_u_i': 'referrer_https%3A%2F%2Fwww.expo2025.or.jp%2F~cookiedomain_expo2025.or.jp',
    'QueueITAccepted-SDFrts345E-V3_gpreservationa': 'EventId%3Dgpreservationa%26QueueId%3Dundefined%26RedirectType%3Dsafetynet%26IssueTime%3D1759704092%26Hash%3D1a504691436d3c64c656e7c8a07ffd73fdc3ee75bd3d240e88aa396cbda96935',
    'QueueITAccepted-SDFrts345E-V3_ticket0001a': 'EventId%3Dticket0001a%26QueueId%3Dundefined%26RedirectType%3Dsafetynet%26IssueTime%3D1759715659%26Hash%3Dbc7123c12a9bab2bbbb5b5fc570a9364bed86d1eb4aa5c3761c3f16655863261',
    'session_id': '6911617e01ec36efe9b328a08ba89fbbb4e928c679cba24f004750cafa3a631e',
    }

In [ ]:
cookies_check_vacancies = {
    'session_id': '451b1632188dff7db5cf695333fef5d0b786ee903360f015857ae9ad3fc2e173'
}

# Define Functions

## Get Vacancies Info

In [ ]:
import requests
#from bs4 import BeautifulSoup as soup
from datetime import date, datetime, timedelta
import pytz
import time

In [ ]:
import json

In [ ]:
headers = {
  'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
  'Accept': 'application/json, text/plain, */*',
  'Accept-Encoding': 'gzip, deflate, br, zstd',
  'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
}

In [ ]:
def get_schedule(month, cookies, headers=headers):
  url = f'https://ticket.expo2025.or.jp/api/d/schedules/2025/{month}'
  response = requests.get(url, headers = headers, cookies=cookies)
  if response.status_code == 200:
    schedule = response.json()
    return schedule
  else:
    raise ValueError(f'Get Schedule Failed: {response.status_code} {response.content}')
    return None

In [ ]:
def get_vacancies_list(month, cookies, target_dates=None, target_timeslots=None):
  schedule = get_schedule(month,cookies)
  if schedule is not None:
    vacancies = [
        {
            'start_time': timeslot_code,
            'gate_type': gate,
            'entrance_date': f'2025{'0'+str(month) if month < 10 else str(month)}{date}'
        }
        # whole month --> date
        for date, daily_schedule in schedule['states'].items()
        if int(daily_schedule['date_state'])<=1 and (target_dates is None or date in target_dates)#(date in target_dates if target_dates is None else True)
        # date --> gate
        for gate, timeslot in daily_schedule.items()
        if gate != 'date_state'
        # gate --> timeslot
        for timeslot_code, details in timeslot.items()
        if details['time_state'] <= 1 and (target_timeslots is None or details['schedule_name'][:-1] in target_timeslots)#(details['schedule_name'][:-1] in target_timeslots if target_timeslots is None else True)
        ]
    return vacancies

In [ ]:
def get_vacancies_list_with_ticket_ids(ticket_ids, month, cookies, target_dates=None, target_timeslots=None):
  schedule = get_schedule(month,cookies)
  if schedule is not None:
    vacancies = [
        {
            'ticket_ids': ticket_ids,
            'start_time': timeslot_code,
            'gate_type': gate,
            'entrance_date': f'2025{'0'+str(month) if month < 10 else str(month)}{date}'
        }
        # whole month --> date
        for date, daily_schedule in schedule['states'].items()
        if int(daily_schedule['date_state'])<=1 and (target_dates is None or date in target_dates)#(date in target_dates if target_dates is None else True)
        # date --> gate
        for gate, timeslot in daily_schedule.items()
        if gate != 'date_state'
        # gate --> timeslot
        for timeslot_code, details in timeslot.items()
        if details['time_state'] <= 1 and (target_timeslots is None or details['schedule_name'][:-1] in target_timeslots)#(details['schedule_name'][:-1] in target_timeslots if target_timeslots is None else True)
        ]
    return vacancies

In [ ]:
def get_vacancies_list_with_reservation_ids(reservation_id, month, cookies, target_dates=None, target_timeslots=None):
  schedule = get_schedule(month,cookies)
  if schedule is not None:
    vacancies = [
        {
            'user_visiting_reservation_ids': reservation_id,
            'start_time': timeslot_code,
            'gate_type': gate,
            'entrance_date': f'2025{'0'+str(month) if month < 10 else str(month)}{date}'
        }
        # whole month --> date
        for date, daily_schedule in schedule['states'].items()
        if int(daily_schedule['date_state'])<=1 and (target_dates is None or date in target_dates)#(date in target_dates if target_dates is None else True)
        # date --> gate
        for gate, timeslot in daily_schedule.items()
        if gate != 'date_state'
        # gate --> timeslot
        for timeslot_code, details in timeslot.items()
        if details['time_state'] <= 1 and (target_timeslots is None or details['schedule_name'][:-1] in target_timeslots)#(details['schedule_name'][:-1] in target_timeslots if target_timeslots is None else True)
        ]
    return vacancies

## Make Reservation

Post Request
request url: https://ticket.expo2025.or.jp/api/d/user_visiting_reservations  
request payload: {ticket_ids: ["PJGAPPG5FS"], start_time: "1100", gate_type: "1", entrance_date: "20251009"}


No Vacancies
status code: 422 (Unprocessable Content)  
response: {"error":{"name":"stock_not_available","message":"在庫の取得に失敗しました。"}}  


In [ ]:
def get_payload_json(ticket_ids, vacancies):
  sorted_vacancies = sorted(vacancies, key=lambda x: (x['start_time'],x['gate_type']*-1))
  for timeslot in sorted_vacancies:
    timeslot.update({'ticket_ids': ticket_ids})
  return sorted_vacancies

In [ ]:
def grab_vacancies(payload, cookies):
  url = 'https://ticket.expo2025.or.jp/api/d/user_visiting_reservations'
  response = requests.post(url, cookies=cookies, json=payload)
  print(f'{response.status_code}, {response.reason} [{datetime.now(pytz.timezone('Asia/Hong_Kong'))}]')
  print(response.content)
  status_code = response.status_code
  return status_code

In [ ]:
def change_reservation(payload, cookies):
  url = 'https://ticket.expo2025.or.jp/api/d/user_visiting_reservations'
  response = requests.put(url, cookies=cookies, json=payload)
  print(f'{response.status_code}, {response.reason} [{datetime.now(pytz.timezone('Asia/Hong_Kong'))}]')
  print(response.content)
  status_code = response.status_code
  return status_code

In [ ]:
def check_and_grab(ticket_ids, month, cookies, cookies_check_vacancies, target_dates=None, target_timeslots=None, run_time = 1):
  status_code = 0
  error_count = 0
  get_vacancies_error = 0
  gotcha = False
  previous_payloads = []
  stop_time = datetime.now(pytz.timezone('Asia/Hong_Kong')) + timedelta(minutes=run_time)

  # use other account (hence difference cookies) for frequently checking vacancies
  # cookies_check_vacancies = cookies if cookies_check_vacancies is None else cookies_check_vacancies

  while datetime.now(pytz.timezone('Asia/Hong_Kong')) <= stop_time:

    try:
      payloads = get_vacancies_list_with_ticket_ids(ticket_ids, month, cookies_check_vacancies, target_dates=target_dates, target_timeslots=target_timeslots)
      get_vacancies_error = 0
    except ValueError:
      get_vacancies_error += 1
      if get_vacancies_error >5:
        print('Continuous Vacancies Error: Break.')
        break
      else:
        print(f'Vancancies Error: {get_vacancies_error}')
        time.sleep(5)
      continue

    if len(payloads) == 0 or previous_payloads == payloads: # if previous payloads = current payloads : not yet updated
      #print(f'Full [{datetime.now(pytz.timezone('Asia/Hong_Kong'))}]')
      continue

    for payload in payloads:
      status_code = grab_vacancies(payload, cookies)
      print(f'{status_code} {payload}')
      if status_code == 200:
        gotcha = True
        break
      elif status_code == 422:
        error_count = 0 # reset error count --> not continuous error
        time.sleep(2)
        continue
      else:
        print(f'Unexpected status code {status_code}.')
        error_count += 1
        if error_count > 2:
          break
        time.sleep(1)
    if gotcha == True:
      break
    previous_payloads = payloads

In [ ]:
def check_and_change_reservation(reservation_id, month, cookies, cookies_check_vacancies, target_dates=None, target_timeslots=None, run_time = 1):
  status_code = 0
  error_count = 0
  get_vacancies_error = 0
  gotcha = False
  previous_payloads = []
  stop_time = datetime.now(pytz.timezone('Asia/Hong_Kong')) + timedelta(minutes=run_time)

  # use other account (hence difference cookies) for frequently checking vacancies
  # cookies_check_vacancies = cookies if cookies_check_vacancies is None else cookies_check_vacancies

  while datetime.now(pytz.timezone('Asia/Hong_Kong')) <= stop_time:

    try:
      payloads = get_vacancies_list_with_reservation_ids(reservation_id, month, cookies_check_vacancies, target_dates=target_dates, target_timeslots=target_timeslots)
      get_vacancies_error = 0
    except ValueError:
      get_vacancies_error += 1
      if get_vacancies_error >5:
        print('Continuous Vacancies Error: Break.')
        break
      else:
        print(f'Vancancies Error: {get_vacancies_error}')
        time.sleep(5)
      continue

    if len(payloads) == 0 or previous_payloads == payloads: # if previous payloads = current payloads : not yet updated
      #print(f'Full [{datetime.now(pytz.timezone('Asia/Hong_Kong'))}]- {payloads}')
      continue

    for payload in payloads:
      status_code = change_reservation(payload, cookies)
      print(f'{status_code} {payload}')
      if status_code == 200:
        gotcha = True
        break
      elif status_code == 422:
        error_count = 0 # reset error count --> not continuous error
        time.sleep(2)
        continue
      else:
        print(f'Unexpected status code {status_code}.')
        error_count += 1
        if error_count > 2:
          break
        time.sleep(1)
    if gotcha == True:
      break
    previous_payloads = payloads

# Run

Ticket to be changed slot
| Ticket ID | Current Entrance Slot | Reservation ID | Expo Ac |
|---|---|---|---|
| QVVJCYQJUG | 8 Oct 12:00PM | 92604867 | SeventhExpo2025 |
| 7QAU3G7244 | 8 Oct 10:00AM | 21032676 | SeventhExpo2025 |



In [ ]:
ticket_ids = ["QVVJCYQJUG"]
reservation_id = [92604867]
month = 10
target_dates=['08']
target_timeslots=['9:00']

check_and_change_reservation(
    reservation_id,
    month,
    cookies,
    cookies_check_vacancies,
    target_dates=target_dates,
    target_timeslots=target_timeslots,
    run_time = 135
    )

KeyboardInterrupt: 